In [15]:
import pandas as pd
import yfinance as yf
from xgboost import XGBRegressor
import joblib
from optuna import create_study
from optuna_integration import XGBoostPruningCallback

def Dataset(ticker, start, end, days_list):
    try:
        df = yf.download(ticker, start=start, end=end)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None, None
    
    if df.empty:
        print("No data returned.")
        return None, None
    
    df = df[['Adj Close']]
    df.columns = ['adjclose']
    
    features = ['adjclose']
    for day in days_list:
        df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
        features.append(f'adjclose_{day}d')

    df = df.dropna()
    df['target'] = df['adjclose'].shift(-1)
    df = df.dropna()
    
    return df, features

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    
    df, features = Dataset(ticker='NFLX', start='2020-01-01', end='2024-05-14', days_list=[1, 2, 3, 5, 10])
    if df is None or features is None:
        return float('inf')
    
    n = int(len(df) * 0.8)
    train = df[:n]
    test = df[n:]
    
    X_train, y_train = train[features], train['target']
    X_test, y_test = test[features], test['target']
    
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=10, callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    
    preds = model.predict(X_test)
    rmse = ((preds - y_test) ** 2).mean() ** 0.5
    return rmse

def XGBoostTimeSeriesForecastingModelOptuna(df, features, label, perc):
    study = create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_params = study.best_params
    
    n = int(len(df) * (1 - perc))
    train = df[:n]
    test = df[n:]

    X_train, y_train = train[features], train[label]
    X_test, y_test = test[features], test[label]

    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

    joblib.dump(model, 'XGBoostTimeSeriesForecastingModel.pkl')
    
    return None

def predict_prices(ticker, start_date, end_date, days_list):
    df, features = Dataset(ticker=ticker, start=start_date, end=end_date, days_list=days_list)
    
    if df is not None and features is not None:
        XGBoostTimeSeriesForecastingModelOptuna(df, features, label='target', perc=0.2)
        
        model = joblib.load('XGBoostTimeSeriesForecastingModel.pkl')
        
        X = df[features]
        df['predicted_price'] = model.predict(X)
        
        next_day_features = features.copy()
        next_day = df.iloc[[-1]][next_day_features]
        predicted_next_day_price = model.predict(next_day)[0]
        
        return predicted_next_day_price
    else:
        return None

predicted_price = predict_prices(ticker='NFLX', start_date='2020-01-01', end_date='2024-05-14', days_list=[1, 2, 3, 5, 10])
if predicted_price is not None:
    print("Predicted price for MAY 15, 2024:", predicted_price)


[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: 

[0]	validation_0-rmse:127.03304	validation_1-rmse:107.63266
[1]	validation_0-rmse:125.26933	validation_1-rmse:106.17270
[2]	validation_0-rmse:123.53069	validation_1-rmse:104.73257
[3]	validation_0-rmse:121.81709	validation_1-rmse:103.32080
[4]	validation_0-rmse:120.12757	validation_1-rmse:101.92160
[5]	validation_0-rmse:118.46212	validation_1-rmse:100.54262
[6]	validation_0-rmse:116.82068	validation_1-rmse:99.19100
[7]	validation_0-rmse:115.20236	validation_1-rmse:97.85301
[8]	validation_0-rmse:113.60708	validation_1-rmse:96.53381
[9]	validation_0-rmse:112.03451	validation_1-rmse:95.22823
[10]	validation_0-rmse:110.48453	validation_1-rmse:93.95177
[11]	validation_0-rmse:108.95640	validation_1-rmse:92.68357
[12]	validation_0-rmse:107.45007	validation_1-rmse:91.43392
[13]	validation_0-rmse:105.96541	validation_1-rmse:90.21273
[14]	validation_0-rmse:104.50201	validation_1-rmse:88.99881
[15]	validation_0-rmse:103.05913	validation_1-rmse:87.80728
[16]	validation_0-rmse:101.63676	validation_


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

[0]	validation_0-rmse:125.80097	validation_1-rmse:106.61658
[1]	validation_0-rmse:122.85211	validation_1-rmse:104.17586
[2]	validation_0-rmse:119.97409	validation_1-rmse:101.79251
[3]	validation_0-rmse:117.16579	validation_1-rmse:99.47966
[4]	validation_0-rmse:114.42458	validation_1-rmse:97.21327
[5]	validation_0-rmse:111.74929	validation_1-rmse:94.98969
[6]	validation_0-rmse:109.13826	validation_1-rmse:92.83275
[7]	validation_0-rmse:106.59050	validation_1-rmse:90.73679
[8]	validation_0-rmse:104.10420	validation_1-rmse:88.67380
[9]	validation_0-rmse:101.67711	validation_1-rmse:86.67904
[10]	validation_0-rmse:99.30695	validation_1-rmse:84.53055
[11]	validation_0-rmse:96.99462	validation_1-rmse:82.63202
[12]	validation_0-rmse:94.73733	validation_1-rmse:80.76701
[13]	validation_0-rmse:92.53401	validation_1-rmse:78.82314
[14]	validation_0-rmse:90.38286	validation_1-rmse:77.06291
[15]	validation_0-rmse:88.28582	validation_1-rmse:75.34901
[16]	validation_0-rmse:86.23506	validation_1-rmse:73.


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

...

[0]	validation_0-rmse:92.85022	validation_1-rmse:79.50434
[1]	validation_0-rmse:67.18208	validation_1-rmse:58.51746
[2]	validation_0-rmse:48.88237	validation_1-rmse:42.13870
[3]	validation_0-rmse:35.88869	validation_1-rmse:32.02845
[4]	validation_0-rmse:26.73243	validation_1-rmse:25.05087



/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/1895165916.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

[5]	validation_0-rmse:20.34747	validation_1-rmse:20.27020
[6]	validation_0-rmse:15.92960	validation_1-rmse:17.66331
[7]	validation_0-rmse:12.94191	validation_1-rmse:16.13830
[8]	validation_0-rmse:10.89044	validation_1-rmse:15.29633
[9]	validation_0-rmse:9.44166	validation_1-rmse:14.48294
[10]	validation_0-rmse:8.52022	validation_1-rmse:14.41389
[11]	validation_0-rmse:7.79913	validation_1-rmse:14.26139
[12]	validation_0-rmse:7.36532	validation_1-rmse:14.18355
[13]	validation_0-rmse:7.02220	validation_1-rmse:14.16458
[14]	validation_0-rmse:6.73398	validation_1-rmse:14.12488
[15]	validation_0-rmse:6.55286	validation_1-rmse:14.15221
[16]	validation_0-rmse:6.38259	validation_1-rmse:14.32179
[17]	validation_0-rmse:6.26174	validation_1-rmse:14.46208
[18]	validation_0-rmse:6.00332	validation_1-rmse:14.37674
[19]	validation_0-rmse:5.89118	validation_1-rmse:14.38340
[20]	validation_0-rmse:5.74340	validation_1-rmse:14.38069
[21]	validation_0-rmse:5.67663	validation_1-rmse:14.38034
[22]	validation

[I 2024-05-22 19:17:45,585] Trial 99 finished with value: 14.124885023818976 and parameters: {'n_estimators': 42, 'learning_rate': 0.28816238999461663}. Best is trial 84 with value: 12.938391105295496.


[0]	validation_0-rmse:91.61067	validation_1-rmse:78.48761
[1]	validation_0-rmse:65.42438	validation_1-rmse:57.07237
[2]	validation_0-rmse:47.01630	validation_1-rmse:40.60125
[3]	validation_0-rmse:34.13861	validation_1-rmse:30.65383
[4]	validation_0-rmse:25.18845	validation_1-rmse:23.71422
[5]	validation_0-rmse:19.06373	validation_1-rmse:19.23363
[6]	validation_0-rmse:14.91969	validation_1-rmse:16.57853
[7]	validation_0-rmse:12.09227	validation_1-rmse:14.94902
[8]	validation_0-rmse:10.28567	validation_1-rmse:14.02826
[9]	validation_0-rmse:9.10363	validation_1-rmse:13.36754
[10]	validation_0-rmse:8.33266	validation_1-rmse:13.21070
[11]	validation_0-rmse:7.73027	validation_1-rmse:13.05988
[12]	validation_0-rmse:7.33581	validation_1-rmse:13.03061
[13]	validation_0-rmse:7.01713	validation_1-rmse:12.94222
[14]	validation_0-rmse:6.66968	validation_1-rmse:12.93839
[15]	validation_0-rmse:6.40960	validation_1-rmse:13.12187
[16]	validation_0-rmse:6.19872	validation_1-rmse:13.16039
[17]	validation

In [21]:
import pandas as pd
import yfinance as yf
from xgboost import XGBRegressor
import joblib
from optuna import create_study
from optuna_integration import XGBoostPruningCallback

def Dataset(ticker, start, end, days_list):
    try:
        df = yf.download(ticker, start=start, end=end)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None, None
    
    if df.empty:
        print("No data returned.")
        return None, None
    
    df = df[['Adj Close']]
    df.columns = ['adjclose']
    
    features = ['adjclose']
    for day in days_list:
        df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
        features.append(f'adjclose_{day}d')

    df = df.dropna()
    df['target'] = df['adjclose'].shift(-1)
    df = df.dropna()
    
    return df, features

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    
    df, features = Dataset(ticker='NFLX', start='2020-01-01', end='2024-05-14', days_list=[1, 2, 3, 5, 10])
    if df is None or features is None:
        return float('inf')
    
    n = int(len(df) * 0.8)
    train = df[:n]
    test = df[n:]
    
    X_train, y_train = train[features], train['target']
    X_test, y_test = test[features], test['target']
    
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=10, callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    
    preds = model.predict(X_test)
    rmse = ((preds - y_test) ** 2).mean() ** 0.5
    return rmse

def XGBoostTimeSeriesForecastingModelOptuna(df, features, label, perc):
    study = create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_params = study.best_params
    
    n = int(len(df) * (1 - perc))
    train = df[:n]
    test = df[n:]

    X_train, y_train = train[features], train[label]
    X_test, y_test = test[features], test[label]

    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

    joblib.dump(model, 'XGBoostTimeSeriesForecastingModel.pkl')
    
    return None

def predict_prices(ticker, start_date, end_date, days_list):
    df, features = Dataset(ticker=ticker, start=start_date, end=end_date, days_list=days_list)
    
    if df is not None and features is not None:
        XGBoostTimeSeriesForecastingModelOptuna(df, features, label='target', perc=0.2)
        
        model = joblib.load('XGBoostTimeSeriesForecastingModel.pkl')
        
        X = df[features]
        df['predicted_price'] = model.predict(X)
        
        next_day_features = features.copy()
        next_day = df.iloc[[-1]][next_day_features]
        predicted_next_day_price = model.predict(next_day)[0]
        
        return predicted_next_day_price
    else:
        return None

predicted_price = predict_prices(ticker='TSLA', start_date='2020-01-01', end_date='2024-05-14', days_list=[1, 2, 3, 5, 10])
if predicted_price is not None:
    print("Predicted price for MAY 15, 2024:", predicted_price)


[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: 

[0]	validation_0-rmse:100.73337	validation_1-rmse:85.97709
[1]	validation_0-rmse:78.93772	validation_1-rmse:68.07302
[2]	validation_0-rmse:61.97151	validation_1-rmse:52.82979
[3]	validation_0-rmse:48.82099	validation_1-rmse:42.27295
[4]	validation_0-rmse:38.67706	validation_1-rmse:34.31287
[5]	validation_0-rmse:30.86384	validation_1-rmse:28.16543
[6]	validation_0-rmse:24.84385	validation_1-rmse:23.81932
[7]	validation_0-rmse:20.27921	validation_1-rmse:20.43030
[8]	validation_0-rmse:16.81772	validation_1-rmse:18.26848
[9]	validation_0-rmse:14.25075	validation_1-rmse:16.65118
[10]	validation_0-rmse:12.37211	validation_1-rmse:15.74252
[11]	validation_0-rmse:10.90840	validation_1-rmse:15.01855
[12]	validation_0-rmse:9.79540	validation_1-rmse:14.55312
[13]	validation_0-rmse:9.05486	validation_1-rmse:14.23490
[14]	validation_0-rmse:8.43461	validation_1-rmse:14.05271
[15]	validation_0-rmse:7.99590	validation_1-rmse:13.93144
[16]	validation_0-rmse:7.58405	validation_1-rmse:13.81313
[17]	valida


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

[0]	validation_0-rmse:113.44371	validation_1-rmse:96.43326
[1]	validation_0-rmse:99.94734	validation_1-rmse:85.24167
[2]	validation_0-rmse:88.10783	validation_1-rmse:75.55384
[3]	validation_0-rmse:77.69003	validation_1-rmse:66.14671
[4]	validation_0-rmse:68.56724	validation_1-rmse:58.66767
[5]	validation_0-rmse:60.54806	validation_1-rmse:52.08550
[6]	validation_0-rmse:53.52015	validation_1-rmse:46.38402
[7]	validation_0-rmse:47.35846	validation_1-rmse:41.46690
[8]	validation_0-rmse:41.96097	validation_1-rmse:36.95920
[9]	validation_0-rmse:37.25053	validation_1-rmse:33.21172
[10]	validation_0-rmse:33.11067	validation_1-rmse:29.77981
[11]	validation_0-rmse:29.50349	validation_1-rmse:27.08227
[12]	validation_0-rmse:26.36052	validation_1-rmse:24.79090
[13]	validation_0-rmse:23.62947	validation_1-rmse:22.68697
[14]	validation_0-rmse:21.26209	validation_1-rmse:21.03829
[15]	validation_0-rmse:19.21009	validation_1-rmse:19.62629
[16]	validation_0-rmse:17.41685	validation_1-rmse:18.53898
[17]	v


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

...

[9]	validation_0-rmse:9.12788	validation_1-rmse:13.51008
[10]	validation_0-rmse:8.32194	validation_1-rmse:13.44064
[11]	validation_0-rmse:7.71695	validation_1-rmse:13.29939
[12]	validation_0-rmse:7.31393	validation_1-rmse:13.24403
[13]	validation_0-rmse:7.08590	validation_1-rmse:13.21357


[I 2024-05-22 19:18:47,740] Trial 96 pruned. Trial was pruned at iteration 14.
[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var

[0]	validation_0-rmse:92.08523	validation_1-rmse:78.87683
[1]	validation_0-rmse:66.09327	validation_1-rmse:57.61467
[2]	validation_0-rmse:47.72337	validation_1-rmse:41.16963
[3]	validation_0-rmse:34.79731	validation_1-rmse:31.13231
[4]	validation_0-rmse:25.76160	validation_1-rmse:24.07724
[5]	validation_0-rmse:19.55926	validation_1-rmse:19.64144
[6]	validation_0-rmse:15.29399	validation_1-rmse:17.09988
[7]	validation_0-rmse:12.46437	validation_1-rmse:15.29541
[8]	validation_0-rmse:10.59779	validation_1-rmse:14.55083
[9]	validation_0-rmse:9.28249	validation_1-rmse:14.00987
[10]	validation_0-rmse:8.42813	validation_1-rmse:13.82495
[11]	validation_0-rmse:7.72927	validation_1-rmse:13.83803
[12]	validation_0-rmse:7.34539	validation_1-rmse:13.93153
[13]	validation_0-rmse:6.93727	validation_1-rmse:13.95187
[14]	validation_0-rmse:6.67335	validation_1-rmse:13.91710


[I 2024-05-22 19:18:47,815] Trial 98 pruned. Trial was pruned at iteration 15.
[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/2734201084.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var

[0]	validation_0-rmse:62.94530	validation_1-rmse:28.25229
[1]	validation_0-rmse:44.98380	validation_1-rmse:20.45116
[2]	validation_0-rmse:32.39036	validation_1-rmse:15.17901
[3]	validation_0-rmse:23.62043	validation_1-rmse:11.64151
[4]	validation_0-rmse:17.56234	validation_1-rmse:9.75271
[5]	validation_0-rmse:13.41710	validation_1-rmse:8.58620
[6]	validation_0-rmse:10.59915	validation_1-rmse:7.99163
[7]	validation_0-rmse:8.68569	validation_1-rmse:7.67076
[8]	validation_0-rmse:7.41497	validation_1-rmse:7.59316
[9]	validation_0-rmse:6.61733	validation_1-rmse:7.54102
[10]	validation_0-rmse:6.04642	validation_1-rmse:7.52020
[11]	validation_0-rmse:5.62384	validation_1-rmse:7.54121
[12]	validation_0-rmse:5.33373	validation_1-rmse:7.59453
[13]	validation_0-rmse:5.09187	validation_1-rmse:7.59595
[14]	validation_0-rmse:4.90607	validation_1-rmse:7.60767
[15]	validation_0-rmse:4.75971	validation_1-rmse:7.61544
[16]	validation_0-rmse:4.60824	validation_1-rmse:7.70486
[17]	validation_0-rmse:4.50935

In [22]:
import pandas as pd
import yfinance as yf
from xgboost import XGBRegressor
import joblib
from optuna import create_study
from optuna_integration import XGBoostPruningCallback

def Dataset(ticker, start, end, days_list):
    try:
        df = yf.download(ticker, start=start, end=end)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None, None
    
    if df.empty:
        print("No data returned.")
        return None, None
    
    df = df[['Adj Close']]
    df.columns = ['adjclose']
    
    features = ['adjclose']
    for day in days_list:
        df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
        features.append(f'adjclose_{day}d')

    df = df.dropna()
    df['target'] = df['adjclose'].shift(-1)
    df = df.dropna()
    
    return df, features

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    
    df, features = Dataset(ticker='NFLX', start='2020-01-01', end='2024-05-14', days_list=[1, 2, 3, 5, 10])
    if df is None or features is None:
        return float('inf')
    
    n = int(len(df) * 0.8)
    train = df[:n]
    test = df[n:]
    
    X_train, y_train = train[features], train['target']
    X_test, y_test = test[features], test['target']
    
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=10, callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    
    preds = model.predict(X_test)
    rmse = ((preds - y_test) ** 2).mean() ** 0.5
    return rmse

def XGBoostTimeSeriesForecastingModelOptuna(df, features, label, perc):
    study = create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_params = study.best_params
    
    n = int(len(df) * (1 - perc))
    train = df[:n]
    test = df[n:]

    X_train, y_train = train[features], train[label]
    X_test, y_test = test[features], test[label]

    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

    joblib.dump(model, 'XGBoostTimeSeriesForecastingModel.pkl')
    
    return None

def predict_prices(ticker, start_date, end_date, days_list):
    df, features = Dataset(ticker=ticker, start=start_date, end=end_date, days_list=days_list)
    
    if df is not None and features is not None:
        XGBoostTimeSeriesForecastingModelOptuna(df, features, label='target', perc=0.2)
        
        model = joblib.load('XGBoostTimeSeriesForecastingModel.pkl')
        
        X = df[features]
        df['predicted_price'] = model.predict(X)
        
        next_day_features = features.copy()
        next_day = df.iloc[[-1]][next_day_features]
        predicted_next_day_price = model.predict(next_day)[0]
        
        return predicted_next_day_price
    else:
        return None

predicted_price = predict_prices(ticker='NKE', start_date='2020-01-01', end_date='2024-05-14', days_list=[1, 2, 3, 5, 10])
if predicted_price is not None:
    print("Predicted price for MAY 15, 2024:", predicted_price)


[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: Set

[0]	validation_0-rmse:123.73490	validation_1-rmse:104.91299
[1]	validation_0-rmse:118.85290	validation_1-rmse:100.87307
[2]	validation_0-rmse:114.16860	validation_1-rmse:96.99561
[3]	validation_0-rmse:109.67417	validation_1-rmse:93.26493
[4]	validation_0-rmse:105.36222	validation_1-rmse:89.71846
[5]	validation_0-rmse:101.22555	validation_1-rmse:86.28664
[6]	validation_0-rmse:97.25284	validation_1-rmse:82.68670
[7]	validation_0-rmse:93.44081	validation_1-rmse:79.56387
[8]	validation_0-rmse:89.78662	validation_1-rmse:76.57970
[9]	validation_0-rmse:86.27290	validation_1-rmse:73.48141
[10]	validation_0-rmse:82.90348	validation_1-rmse:70.70548
[11]	validation_0-rmse:79.67451	validation_1-rmse:68.06488



/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

[0]	validation_0-rmse:125.81487	validation_1-rmse:106.62804
[1]	validation_0-rmse:122.87925	validation_1-rmse:104.19828
[2]	validation_0-rmse:120.01383	validation_1-rmse:101.82534
[3]	validation_0-rmse:117.21750	validation_1-rmse:99.52236
[4]	validation_0-rmse:114.48765	validation_1-rmse:97.26553
[5]	validation_0-rmse:111.82316	validation_1-rmse:95.05073
[6]	validation_0-rmse:109.22238	validation_1-rmse:92.90223
[7]	validation_0-rmse:106.68432	validation_1-rmse:90.81425
[8]	validation_0-rmse:104.20721	validation_1-rmse:88.75888
[9]	validation_0-rmse:101.78882	validation_1-rmse:86.77122
[10]	validation_0-rmse:99.42690	validation_1-rmse:84.63018
[11]	validation_0-rmse:97.12272	validation_1-rmse:82.73794
[12]	validation_0-rmse:94.87274	validation_1-rmse:80.87778
[13]	validation_0-rmse:92.67604	validation_1-rmse:78.94182
[14]	validation_0-rmse:90.53130	validation_1-rmse:77.19376
[15]	validation_0-rmse:88.44088	validation_1-rmse:75.48006
[16]	validation_0-rmse:86.39760	validation_1-rmse:73.


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

...

[0]	validation_0-rmse:91.40090	validation_1-rmse:78.31559
[1]	validation_0-rmse:65.12999	validation_1-rmse:56.83378
[2]	validation_0-rmse:46.70749	validation_1-rmse:40.35270



/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

[3]	validation_0-rmse:33.85199	validation_1-rmse:30.43967
[4]	validation_0-rmse:24.97639	validation_1-rmse:23.54148
[5]	validation_0-rmse:18.90228	validation_1-rmse:19.10451
[6]	validation_0-rmse:14.80825	validation_1-rmse:16.43345
[7]	validation_0-rmse:12.14342	validation_1-rmse:15.06035
[8]	validation_0-rmse:10.31200	validation_1-rmse:14.15727
[9]	validation_0-rmse:9.08918	validation_1-rmse:13.92141
[10]	validation_0-rmse:8.26777	validation_1-rmse:13.61229
[11]	validation_0-rmse:7.68957	validation_1-rmse:13.61688
[12]	validation_0-rmse:7.34148	validation_1-rmse:13.48658
[13]	validation_0-rmse:7.02374	validation_1-rmse:13.40447
[14]	validation_0-rmse:6.71947	validation_1-rmse:13.48113
[15]	validation_0-rmse:6.52295	validation_1-rmse:13.51584
[16]	validation_0-rmse:6.21679	validation_1-rmse:13.54903
[17]	validation_0-rmse:6.00469	validation_1-rmse:13.57593
[18]	validation_0-rmse:5.89001	validation_1-rmse:13.57220
[19]	validation_0-rmse:5.79008	validation_1-rmse:13.57095
[20]	validation

[I 2024-05-22 19:20:36,586] Trial 97 pruned. Trial was pruned at iteration 23.
[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/988769727.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/f

[0]	validation_0-rmse:16.56047	validation_1-rmse:14.34226
[1]	validation_0-rmse:11.87811	validation_1-rmse:10.24358
[2]	validation_0-rmse:8.59077	validation_1-rmse:7.36724
[3]	validation_0-rmse:6.29196	validation_1-rmse:5.47793
[4]	validation_0-rmse:4.70326	validation_1-rmse:4.15322
[5]	validation_0-rmse:3.61442	validation_1-rmse:3.33167
[6]	validation_0-rmse:2.86577	validation_1-rmse:2.83796
[7]	validation_0-rmse:2.37470	validation_1-rmse:2.48895
[8]	validation_0-rmse:2.03328	validation_1-rmse:2.32315
[9]	validation_0-rmse:1.80341	validation_1-rmse:2.23420
[10]	validation_0-rmse:1.66069	validation_1-rmse:2.19356
[11]	validation_0-rmse:1.56547	validation_1-rmse:2.17634
[12]	validation_0-rmse:1.48077	validation_1-rmse:2.16485
[13]	validation_0-rmse:1.41976	validation_1-rmse:2.16072
[14]	validation_0-rmse:1.36396	validation_1-rmse:2.15647
[15]	validation_0-rmse:1.32604	validation_1-rmse:2.15957
[16]	validation_0-rmse:1.29230	validation_1-rmse:2.15813
[17]	validation_0-rmse:1.26566	valida

In [18]:
import pandas as pd
import yfinance as yf
from xgboost import XGBRegressor
import joblib
from optuna import create_study
from optuna_integration import XGBoostPruningCallback

def Dataset(ticker, start, end, days_list):
    try:
        df = yf.download(ticker, start=start, end=end)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None, None
    
    if df.empty:
        print("No data returned.")
        return None, None
    
    df = df[['Adj Close']]
    df.columns = ['adjclose']
    
    features = ['adjclose']
    for day in days_list:
        df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
        features.append(f'adjclose_{day}d')

    df = df.dropna()
    df['target'] = df['adjclose'].shift(-1)
    df = df.dropna()
    
    return df, features

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    
    df, features = Dataset(ticker='NFLX', start='2020-01-01', end='2024-05-14', days_list=[1, 2, 3, 5, 10])
    if df is None or features is None:
        return float('inf')
    
    n = int(len(df) * 0.8)
    train = df[:n]
    test = df[n:]
    
    X_train, y_train = train[features], train['target']
    X_test, y_test = test[features], test['target']
    
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=10, callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    
    preds = model.predict(X_test)
    rmse = ((preds - y_test) ** 2).mean() ** 0.5
    return rmse

def XGBoostTimeSeriesForecastingModelOptuna(df, features, label, perc):
    study = create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_params = study.best_params
    
    n = int(len(df) * (1 - perc))
    train = df[:n]
    test = df[n:]

    X_train, y_train = train[features], train[label]
    X_test, y_test = test[features], test[label]

    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

    joblib.dump(model, 'XGBoostTimeSeriesForecastingModel.pkl')
    
    return None

def predict_prices(ticker, start_date, end_date, days_list):
    df, features = Dataset(ticker=ticker, start=start_date, end=end_date, days_list=days_list)
    
    if df is not None and features is not None:
        XGBoostTimeSeriesForecastingModelOptuna(df, features, label='target', perc=0.2)
        
        model = joblib.load('XGBoostTimeSeriesForecastingModel.pkl')
        
        X = df[features]
        df['predicted_price'] = model.predict(X)
        
        next_day_features = features.copy()
        next_day = df.iloc[[-1]][next_day_features]
        predicted_next_day_price = model.predict(next_day)[0]
        
        return predicted_next_day_price
    else:
        return None

predicted_price = predict_prices(ticker='AAPL', start_date='2020-01-01', end_date='2024-05-14', days_list=[1, 2, 3, 5, 10])
if predicted_price is not None:
    print("Predicted price for MAY 15, 2024:", predicted_price)


[*********************100%%**********************]  1 of 1 completed
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: 

[0]	validation_0-rmse:94.58951	validation_1-rmse:80.93149
[1]	validation_0-rmse:69.69293	validation_1-rmse:60.55041
[2]	validation_0-rmse:51.58037	validation_1-rmse:44.32347
[3]	validation_0-rmse:38.44796	validation_1-rmse:34.03663
[4]	validation_0-rmse:29.02578	validation_1-rmse:26.72738
[5]	validation_0-rmse:22.28810	validation_1-rmse:21.89431
[6]	validation_0-rmse:17.53406	validation_1-rmse:18.69916
[7]	validation_0-rmse:14.18088	validation_1-rmse:16.68277
[8]	validation_0-rmse:11.84856	validation_1-rmse:15.64645
[9]	validation_0-rmse:10.27461	validation_1-rmse:15.05439
[10]	validation_0-rmse:9.20030	validation_1-rmse:14.73427
[11]	validation_0-rmse:8.37336	validation_1-rmse:14.38295
[12]	validation_0-rmse:7.78150	validation_1-rmse:14.25794
[13]	validation_0-rmse:7.35011	validation_1-rmse:14.12861
[14]	validation_0-rmse:7.01459	validation_1-rmse:14.11402
[15]	validation_0-rmse:6.74197	validation_1-rmse:14.04593
[16]	validation_0-rmse:6.48740	validation_1-rmse:14.24651
[17]	validatio

[I 2024-05-22 19:17:55,809] Trial 0 finished with value: 14.045930838051865 and parameters: {'n_estimators': 63, 'learning_rate': 0.274186349081349}. Best is trial 0 with value: 14.045930838051865.
[*********************100%%**********************]  1 of 1 completed

[0]	validation_0-rmse:117.60755	validation_1-rmse:99.86289
[1]	validation_0-rmse:107.39298	validation_1-rmse:91.42548



/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

...

[8]	validation_0-rmse:11.23350	validation_1-rmse:15.14076
[9]	validation_0-rmse:9.72728	validation_1-rmse:14.65935
[10]	validation_0-rmse:8.79875	validation_1-rmse:14.11468
[11]	validation_0-rmse:8.09203	validation_1-rmse:13.81892
[12]	validation_0-rmse:7.57248	validation_1-rmse:13.71534
[13]	validation_0-rmse:7.20918	validation_1-rmse:13.72672
[14]	validation_0-rmse:6.89533	validation_1-rmse:13.67133
[15]	validation_0-rmse:6.69196	validation_1-rmse:13.66133
[16]	validation_0-rmse:6.44059	validation_1-rmse:13.74555
[17]	validation_0-rmse:6.26150	validation_1-rmse:13.76839
[18]	validation_0-rmse:6.06417	validation_1-rmse:13.73484


[I 2024-05-22 19:18:00,098] Trial 98 pruned. Trial was pruned at iteration 19.
[*********************100%%**********************]  1 of 1 completed

[0]	validation_0-rmse:120.88726	validation_1-rmse:102.56556



/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'adjclose_{day}d'] = df['adjclose'].shift(day)
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_47494/3751722141.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

[0]	validation_0-rmse:21.70316	validation_1-rmse:35.96571
[1]	validation_0-rmse:15.50627	validation_1-rmse:26.70614
[2]	validation_0-rmse:11.14133	validation_1-rmse:20.60960
[3]	validation_0-rmse:8.09318	validation_1-rmse:15.64256
[4]	validation_0-rmse:5.96474	validation_1-rmse:12.04715
[5]	validation_0-rmse:4.50015	validation_1-rmse:9.58783
[6]	validation_0-rmse:3.52060	validation_1-rmse:7.93899
[7]	validation_0-rmse:2.86885	validation_1-rmse:6.75586
[8]	validation_0-rmse:2.44373	validation_1-rmse:5.77662
[9]	validation_0-rmse:2.16189	validation_1-rmse:5.11963
[10]	validation_0-rmse:1.97421	validation_1-rmse:4.60678
[11]	validation_0-rmse:1.83822	validation_1-rmse:4.24596
[12]	validation_0-rmse:1.75448	validation_1-rmse:3.97023
[13]	validation_0-rmse:1.68811	validation_1-rmse:3.88893
[14]	validation_0-rmse:1.62774	validation_1-rmse:3.72789
[15]	validation_0-rmse:1.57379	validation_1-rmse:3.70802
[16]	validation_0-rmse:1.53847	validation_1-rmse:3.67125
[17]	validation_0-rmse:1.51058	va